# Setting up the containers for the other notebooks in this folder

To be able to follow along with the rest of the notebooks, you will need to set up two containers. 

This notebook will enable you to do that.

You will need to have Docker installed. The image is based on the SQL Server 2019 image so you will need to have docker set to use Linux Containers. Right click on the Docker icon in the notification area and if it says "Switch to Linux Containers" click to switch and wait for Docker to restart.  

![Switch To Linux Containers](.\images\switchtolinuxcontainers.png )

You will be able to run all of the code in the notebooks by creating the folder, credential and containers in this notebook and then you can click on the play button in each code block to run the code. Note - There are a few code blocks with the results already included which should not be run. They are to show you the results of a command that cannot be run against containers (setting up configuration for domain accounts for example)

## Create the folders and the credential

The code below will create a directory called dbatools-demo in your Documents folder and save a credential file for logging into the containers. You can alter the directory created by altering the `$FolderPath`  

The code has been collapsed but you can view it using the view menu in the lab or by clicking the 3 ellipses

In [3]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbatoolsdemo'

########################################################
Write-Output "Creating Directory $FolderPath"
if(Test-Path $FolderPath){
    Write-Output "Path $FolderPath exists already"
}else {
 $null =    New-Item $FolderPath -ItemType Directory
}

Write-Output "Creating Directory $FolderPath\SQL1"
if(Test-Path "$FolderPath\SQL1"){
    Write-Output "Directory SQL1 exists already"
    Get-ChildItem "$FolderPath\SQL1" -Recurse| Remove-Item -Force
}else {
$null = New-Item "$FolderPath\SQL1"-ItemType Directory
}
Write-Output "Creating File $FolderPath\SQL1\dummyfile.txt"
if(Test-Path "$FolderPath\SQL1\dummyfile.txt"){
    Write-Output "dummyfile.txt exists already"
}else {
$null =  New-Item "$FolderPath\SQL1\dummyfile.txt" -ItemType file
}

Write-Output "Creating Directory $FolderPath\SQL2"
if(Test-Path "$FolderPath\SQL2"){
    Write-Output "Directory SQL2 exists already"
    Get-ChildItem "$FolderPath\SQL2" -Recurse| Remove-Item -Force
}else {
 $null =    New-Item "$FolderPath\SQL2"-ItemType Directory
}
Write-Output "Creating File $FolderPath\SQL2\dummyfile.txt"
if(Test-Path "$FolderPath\SQL2\dummyfile.txt"){
    Write-Output "dummyfile.txt exists already"
}else {
$null =  New-Item "$FolderPath\SQL2\dummyfile.txt" -ItemType file
}

Write-Output "Creating a credential file for the containers - Please don't do this in production"

$sqladminPassword = ConvertTo-SecureString 'dbatools.IO' -AsPlainText -Force 
$cred = New-Object System.Management.Automation.PSCredential ('sqladmin', $sqladminpassword)
$Cred | Export-Clixml -Path $FolderPath\sqladmin.cred
Write-Output "Credential file created"

Write-Output "Setting the docker-compose files values"
$dockercompose = @"
version: '3.7'
services:
    SQL2019:
        image: sqldbawithabeard/dbachecksdemo
        ports:  
          - "15592:1433"
        networks: 
          - BeardsMagicNetwork
        volumes: 
          - __ReplaceMe__:/var/opt/mssql/backups
    SQL2019-1:
        image: sqldbawithabeard/dbachecksdemo
        ports:  
          - "15593:1433"
        networks: 
          - BeardsMagicNetwork
        volumes: 
          - __ReplaceMe__:/var/opt/mssql/backups
networks: 
  BeardsMagicNetwork:

"@
Set-Location $FolderPath

$dockercompose = $dockercompose -replace '__ReplaceME__' , $FolderPath
# $dockercompose
$dockercompose | Set-Content .\docker-compose.yml



Creating Directory C:\Users\mrrob\Documents\dbatoolsdemo
Creating Directory C:\Users\mrrob\Documents\dbatoolsdemo\SQL1
Creating File C:\Users\mrrob\Documents\dbatoolsdemo\SQL1\dummyfile.txt
Creating Directory C:\Users\mrrob\Documents\dbatoolsdemo\SQL2
Creating File C:\Users\mrrob\Documents\dbatoolsdemo\SQL2\dummyfile.txt
Creating a credential file for the containers - Please don't do this in production
Credential file created
Setting the docker-compose files values


## Containers

We are going to create two SQL 2019 containers using the sqldbawithabeard\dbachecks [image from the Docker Hub](https://hub.docker.com/repository/docker/sqldbawithabeard/dbachecksdemo). 

The first time it is going to pull the image sqldbawithabeard/dbachecksdemo from the Docker Hub. If you wish to do this first, you can run the cell below



In [0]:
docker pull sqldbawithabeard/dbachecksdemo

Now we can start the containers with the code below

In [5]:
docker-compose up -d

Creating network "dbatoolsdemo_BeardsMagicNetwork" with the default driver
Creating dbatoolsdemo_SQL2019_1 ... 
Creating dbatoolsdemo_SQL2019-1_1 ... 
Creating dbatoolsdemo_SQL2019-1_1 ... doneCreating dbatoolsdemo_SQL2019_1   ... done

All being well, you will have something that looks like


![DockerCompose](.\images\containers.png )

If you get an error you might need to add the user you have shared your drives with Docker read and execute permissions to the \Documents\ directory in your user profile   
More details https://docs.docker.com/docker-for-windows/  

If you need to install dbatools please continue - otherwise look at the other Notebooks in the folder

In [0]:
if(Get-Module dbatools -ListAvailable){
Write-Output "Updating dbatools"
Update-Module dbatools
}else {
Write-Output "Installing dbatools in your user profile"
Install-Module dbatools -Scope CurrentUser
}
Import-Module dbatools

Now that is done, we can make a connection to our instances and see if we can connect to them.

We are going to use `Connect-DBaInstance` to do this and we will use the containers that we have created and because we need to use SQL Authentication, we will use a credential that we have saved to disk using `Export-CliXML` in one of the celss above.

It is **Important** to state that this is not a production secure solution and should not be used as a means of accessing any secure system.

There will be no output from the code below but it will have created a connection to the two containers

In [101]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbachecks'
$SqlInstances = 'localhost,15592','localhost,15593'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred

$SQL1 = Connect-DbaInstance -SqlInstance $SqlInstances[0] -SqlCredential $SqlCredential
$SQL2 = Connect-DbaInstance -SqlInstance $SqlInstances[1] -SqlCredential $SqlCredential

# Databases

Lets take a look at the databases that are available on the containers


In [102]:
Get-DbaDatabase -SqlInstance $SQL1,$SQL2 -SqlCredential $SqlCredential | Format-Table

We have some information about each of the databases, such as their name, status,RecoveryModel, LogReUseWait status and size by default.

If we only wanted to look at the user databases on the instance we can use 

In [90]:
Get-DbaDatabase -SqlInstance $SQL1,$SQL2 -SqlCredential $SqlCredential -ExcludeSystem | Format-Table

If we want to get a database by name

In [91]:
Get-DbaDatabase -SqlInstance $SQL1,$SQL2 -SqlCredential $SqlCredential -Database NorthWind 

# Clean Up

The code below will clean up the files that have been created and remove the docker containers but not the image so tha tyou can run this notebook again without having to download the image again

In [106]:
docker-compose down 
$FolderPath = $Env:USERPROFILE + '\Documents\dbatoolsdemo'
Remove-Item $FolderPath -Recurse -Force